In [6]:
import pandas as pd
import numpy as np

In [7]:
import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [8]:
data = pd.read_csv("bbc-text.csv")

In [9]:
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [10]:
data.shape

(2225, 2)

In [11]:
data.head(7)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...


In [12]:
data['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [13]:

print(len(data)*.8)

1780.0


In [14]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))
data[0:train_size]

Train size: 1780
Test size: 445


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
1775,tech,china blocks google news site china has been...
1776,politics,labour accused of eu propaganda a taxpayer ...
1777,sport,tevez - an argentine in brazil some 65 years a...
1778,business,water firm suez in argentina row a conflict be...


In [15]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [16]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [17]:
train_text.head()

0    tv future in the hands of viewers with home th...
1    worldcom boss  left books alone  former worldc...
2    tigers wary of farrell  gamble  leicester say ...
3    yeading face newcastle in fa cup premiership s...
4    ocean s twelve raids box office ocean s twelve...
Name: text, dtype: object

In [18]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, \
                                              char_level=False)
display(train_text.shape)
display(tokenize.texts_to_matrix(train_text).shape)
display(tokenize.texts_to_matrix(train_text))

(1780,)

(1780, 1000)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)


In [20]:
x_test

array([[0., 1., 1., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [21]:
encoder = LabelEncoder()
encoder.fit(train_cat)

y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)
print(y_train)
display(encoder.fit(train_cat))



[4 0 3 ... 3 0 1]


LabelEncoder()

In [22]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
print(num_classes)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train)

5
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [23]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1780, 1000)
x_test shape: (445, 1000)
y_train shape: (1780, 5)
y_test shape: (445, 5)


In [24]:
batch_size = 32
epochs = 4
drop_ratio = 0.5

In [25]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Dense(212 , activation = 'relu'))
model.add(layers.Activation('relu'))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [26]:
history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=2,validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/4
1602/1602 - 1s - loss: 0.3517 - accuracy: 0.8801 - val_loss: 0.1881 - val_accuracy: 0.9326
Epoch 2/4
1602/1602 - 0s - loss: 0.0186 - accuracy: 0.9963 - val_loss: 0.0990 - val_accuracy: 0.9607
Epoch 3/4
1602/1602 - 0s - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.1039 - val_accuracy: 0.9607
Epoch 4/4
1602/1602 - 0s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0982 - val_accuracy: 0.9663


In [27]:
score = model.evaluate(x_test, y_test,\
                       batch_size=batch_size, verbose=1)


445/445 [==============================] - 0s 153us/sample - loss: 0.1411 - accuracy: 0.9596


In [28]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.14114055934916722
Test accuracy: 0.95955056


In [29]:
# model.predict(x_train[0])
x_train[0].shape

(1000,)

# Hyperparameter tunning

This is a good time to go back and tweak some parameters such as epoch, batch size, dropout ratio, network structure, activation function, and others, to see if you can improve the accuracy.

In this particular case, to make it more challenging, I recommend reducing the max words of the call to keras.preprocessing.text.Tokenizer. This will reduce the number of words for each input sample, thus making it more challenging to accurately predict the category. (Notice that not all hyperparameters are necessarily inside the model. This is one such example.)

The default was up to 1000 words per article. See what happens when you reduce that number to 200 words, or 50 words, or even fewer. As the evaluation accuracy drops, the effects of your hyperparameter tuning will be more pronounced, with successful adjustments making meaningful improvements to the model performance.

To make this process easier to manage, I've encapulated the model definition and training and evaluation calls into one function call. You can add additional parameters as needed.

In [30]:
#by using hyperprameter tunning

def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy', #is use when label is more then 2 otherwise binary_crossentropy for 2 labels
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [31]:
batch_size = 16
epochs = 4
drop_ratio = 0.4
run_experiment(batch_size, epochs, drop_ratio)

batch size: 16, epochs: 4, drop_ratio: 0.4
	Test loss: 0.12294160934646478
	Test accuracy: 0.95730335


In [32]:
text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  

hobbit picture  four years away  lord of the rings ...
Actual label:entertainment
Predicted label: entertainment

game firm holds  cast  auditions video game firm b ...
Actual label:tech
Predicted label: tech

clarke plans migrant point scheme anyone planning  ...
Actual label:politics
Predicted label: politics

radcliffe will compete in london paula radcliffe w ...
Actual label:sport
Predicted label: sport

serena becomes world number two serena williams ha ...
Actual label:sport
Predicted label: sport

ultimate game  award for doom 3 sci-fi shooter doo ...
Actual label:tech
Predicted label: tech

algeria hit by further gas riots algeria suffered  ...
Actual label:business
Predicted label: business

fast lifts rise into record books two high-speed l ...
Actual label:tech
Predicted label: tech

muslim group attacks tv drama 24 a british muslim  ...
Actual label:entertainment
Predicted label: entertainment

us tv special for tsunami relief a us television n ...
Actual label:entertainmen

In [33]:
test_text.iloc[3]

'radcliffe will compete in london paula radcliffe will compete in the flora london marathon this year after deciding her schedule for 2005.  the 31-year-old won the race in 2002 on her marathon debut  defended her title 12 months later and will now seek a third title in the 17 april race.  it doesn t get any better than this for the 25th anniversary   said race director david bedford.  after announcing the greatest men s field ever we now have the greatest women s distance runner ever.  three years ago radcliffe smashed the women s world record in two hours 18 minutes 15 seconds.  the bedford star returned to london 12 months later  lowering her mixed-race world record of 2:17:18  which she set in chicago in october 2003  by one minute 53 secs. radcliffe s career took a setback when she failed to complete the olympic marathon and later dropped out of the athens 10 000m last august. but the 31-year-old bounced back to win the new york marathon in november. radcliffe  however  passed up 

In [48]:
prediction = model.predict(np.array([x_test[15]]))

In [49]:
prediction

array([[3.5525413e-04, 9.9129421e-01, 1.9453793e-04, 9.3295455e-07,
        8.1551038e-03]], dtype=float32)

In [50]:
predicted_label = text_labels[np.argmax(prediction)]
predicted_label

'entertainment'

In [51]:
#print(test_text.iloc[:50], "...")
print('Actual label:' + test_cat.iloc[3])
print("Predicted label: " + predicted_label + "\n") 

Actual label:sport
Predicted label: entertainment



In [52]:
test = "Back when Zoey Deutch was an unknown actress looking for her big break, she got her start in a small role on the Disney channel original show The Suite Life on Deck. The show starred twins Dylan Sprouse and Cole Sprouse, and Zoey was cast as Dylan's characters girlfriend. Anyone would be nervous having their first major onscreen role in front of a mega-successful teen heartthrob, and Zoey wasn't immune to a little bit of fear. She talked with W Magazine about the hilarious experience. I was 3 years old when I decided I wanted to be an actress,I was really getting into my made-up storylines of my Barbie and her dream boat crashing and her breakup with Ken, but my parents were very hesitant about letting a 3-year-old act professionally."
test = "Iran's military leader killed: Qasem Soleimani, top general and one of the most powerful men in Iran, was killed in a drone strike at Baghdad airport early Friday. The deputy head of an Iran-backed Iraqi militia was also killed."
prediction = model.predict(np.array(np.array([x_test[3]])))
display(prediction)
print(x_test[3].shape)
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, \
                                              char_level=False)
display(x_test[3].shape)
display(tokenize.texts_to_matrix(np.array(test).shape))
display(np.array(tokenize.texts_to_matrix(test).shape))

array([[3.2787866e-06, 4.4220570e-03, 4.0368895e-06, 9.9537522e-01,
        1.9548157e-04]], dtype=float32)

(1000,)


(1000,)

array([], shape=(0, 1000), dtype=float64)

array([ 228, 1000])

In [53]:
np.array(test)

array("Iran's military leader killed: Qasem Soleimani, top general and one of the most powerful men in Iran, was killed in a drone strike at Baghdad airport early Friday. The deputy head of an Iran-backed Iraqi militia was also killed.",
      dtype='<U228')

In [54]:
tokenize.fit_on_texts(test) # fit tokenizer to our training text data
sample_train = tokenize.texts_to_matrix(test)
sample_test = tokenize.texts_to_matrix(test)

display(sample_test.shape)
x_test[3].shape

(228, 1000)

(1000,)

In [55]:

# prediction = model.predict(np.array([sample_test]))
np.array(sample_test.shape)

array([ 228, 1000])

In [56]:
prediction = model.predict(tokenize.texts_to_matrix(np.array(test).flatten()))

In [57]:
prediction

array([[0.19570285, 0.21733527, 0.17256777, 0.25107887, 0.1633153 ]],
      dtype=float32)

In [58]:
predicted_label = text_labels[np.argmax(prediction)]
predicted_label

'sport'